In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
games = pd.read_csv('/kaggle/input/chess/games.csv')
games.head()

First look on the data

In [ ]:
games.info(),games.describe()

Looking for null data

In [ ]:
games.isnull().sum()

Checking correlation between variables

In [ ]:
games.corr()

In [ ]:
# Lets Explore Categorial Features
games.select_dtypes(['object']).columns

In [ ]:
games['id'].value_counts() # Which means that players didn't played a lot of games

In [ ]:
games['winner'].value_counts()

In [ ]:
games['victory_status'].value_counts()

In [ ]:
sns.countplot(x='victory_status',data=games,hue='winner',palette='rainbow')
# Clearly See that there is no connection between color of player to winning.

In [ ]:
sns.countplot(x='victory_status',data=games,hue='rated',palette='magma')
# A Strong Connection between rated and resigning - which means that rated players are nobel.

In [ ]:
# Replaceing true/false with numeric
games.replace(False, 0, inplace=True)
games.replace(True, 1, inplace=True)
games.head()

Looking for top openings

In [ ]:
games['opening_name'].value_counts().sort_values(ascending=False).head(20)

Looking for number of turns

In [ ]:
games['turns'].value_counts().sort_values(ascending=False).head()

Average number of turns

In [ ]:
games['turns'].mean()

making our data available to analyze and create a model.

In [ ]:
# Lets try to understand what effects your possibility to win
winner_dummies = pd.get_dummies(games['winner'])
winner_dummies


In [ ]:
games = pd.concat([games,winner_dummies],axis=1)
games.drop('winner',axis=1,inplace=True)
games.drop('draw',axis=1,inplace=True)
games.head()

Exploring the variables through data types

In [ ]:
games.select_dtypes(['object']).columns

In [ ]:
# Id columnm doesn't effect the data 
games.drop('id',axis=1,inplace=True)
games.select_dtypes(['object']).columns

In [ ]:
# Getting dummies for victory status
victorystatus_dummies = pd.get_dummies(games['victory_status'],drop_first=True)
victorystatus_dummies

In [ ]:
games.drop('victory_status',axis=1,inplace=True)
games = pd.concat([games,victorystatus_dummies],axis=1)
games.head()

In [ ]:
# explore object types
games.select_dtypes(['object']).columns

In [ ]:
# Lets Explore top players_move
games['two_moves'] = games['moves'].apply(lambda x: x[:6])
games['two_moves'].value_counts().head(5)

In [ ]:
# Removing unecessary columns
games.drop(['increment_code','white_id','black_id'],axis=1,inplace=True)

In [ ]:
games.drop('moves',axis=1,inplace=True)
games.select_dtypes(['object']).columns

In [ ]:
games['opening_eco'].value_counts().sort_values(ascending=False)
# We can refer to the coding book in the explanation tab, currently can't use it for prediction

In [ ]:
games.drop('opening_eco',axis=1,inplace=True)

In [ ]:
games.drop(['opening_name','two_moves'],axis=1,inplace=True)

In [ ]:
games.select_dtypes(['object']).columns

Evaluating the model

In [ ]:
# View the database again
games.head()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(games.corr(),annot=True,cmap='viridis')

In [ ]:
games.info()

Checking timestamp columns to see if there is relevant data

In [ ]:
games['created_at']

In [ ]:
games['created_at'] = pd.to_datetime(games['created_at'])
games['created_at'].head(5)

In [ ]:
games['last_move_at'] = pd.to_datetime(games['last_move_at'])
games['last_move_at'].head(5)

In [ ]:
games.drop(['last_move_at','created_at'],axis=1,inplace=True)

In [ ]:
games.info()

Trying to predict wether a player is rated or not

In [ ]:
X = games.drop('rated',axis=1)
y = games['rated']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

I'll use random forest classifier at the moment,
large number of estimators relatively 

In [ ]:
rfc = RandomForestClassifier(n_estimators=2000)
rfc.fit(X_train, y_train)
pred_rfc = rfc.predict(X_test)



In [ ]:
print(classification_report(pred_rfc, y_test))
print(confusion_matrix(pred_rfc, y_test))

So Basically we see that our model doesn't know how to explain the data that good
We also saw that there isn't a good coorelation between our variables
we have a good accuracy in f1-score, but we truly miss the precision at some points
sometimes chess may be unpredicatble.
Lets see if we can predict a single data observation

In [ ]:
player = games.drop('rated',axis=1).loc[6]
pred_player = rfc.predict(player.values.reshape(1,-1))
pred_player

Which means that the player predicted as rated.

In [ ]:
games['rated'].loc[6]

the player is rated indeed
So in that case we predicted correctly if the player was rated or not.
conclusions - a better model ? 
increase the number of estimators ?
choose another model predictions ? maybe deep learning neurons method?
maybe trying to understand better correlation between the variables?
maybe chess isn't that predictable?